<a href="https://colab.research.google.com/github/JingHuan921/bert-finetuned-ner/blob/main/Sequence_Tagging_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [2]:
!pip install transformers accelerate evaluate seqeval

!pip install accelerate --upgrade
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.5 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=c45bf782ad0f1d21c84338abc2b5fce5ae0599643d99454536fb87f68e767860
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2a

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
from datasets import load_dataset, load_metric, Dataset, DatasetDict
import evaluate
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification

In [5]:
import tensorflow as tf
from tensorflow import keras

In [6]:
import numpy as np

In [7]:
!pip install --upgrade gensim

word2vec-google-news-300 is a pretrained word embedding model developed using the word2vec algorithm. Here are the key details about it:
- Model Type: Word2Vec
- Training Data: Google News dataset
- Vector Dimensions: 300
- Training Algorithm: Word2Vec (Continuous Bag of Words (CBOW) and Skip-gram models) <br>

The following is to showcase the word embedding of word2vec-google-news-300

In [8]:
import gensim.downloader
# Show all available models in gensim-data
print(list(gensim.downloader.info()['models'].keys()))


['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [9]:
# Download the "glove-twitter-25" embeddings
gnews_vec = gensim.downloader.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [10]:
#take a look at the word embedding of some words
gnews_vec['king']

array([ 1.25976562e-01,  2.97851562e-02,  8.60595703e-03,  1.39648438e-01,
       -2.56347656e-02, -3.61328125e-02,  1.11816406e-01, -1.98242188e-01,
        5.12695312e-02,  3.63281250e-01, -2.42187500e-01, -3.02734375e-01,
       -1.77734375e-01, -2.49023438e-02, -1.67968750e-01, -1.69921875e-01,
        3.46679688e-02,  5.21850586e-03,  4.63867188e-02,  1.28906250e-01,
        1.36718750e-01,  1.12792969e-01,  5.95703125e-02,  1.36718750e-01,
        1.01074219e-01, -1.76757812e-01, -2.51953125e-01,  5.98144531e-02,
        3.41796875e-01, -3.11279297e-02,  1.04492188e-01,  6.17675781e-02,
        1.24511719e-01,  4.00390625e-01, -3.22265625e-01,  8.39843750e-02,
        3.90625000e-02,  5.85937500e-03,  7.03125000e-02,  1.72851562e-01,
        1.38671875e-01, -2.31445312e-01,  2.83203125e-01,  1.42578125e-01,
        3.41796875e-01, -2.39257812e-02, -1.09863281e-01,  3.32031250e-02,
       -5.46875000e-02,  1.53198242e-02, -1.62109375e-01,  1.58203125e-01,
       -2.59765625e-01,  

In [11]:
def load_dataset(file):
    with open (file, 'r') as file_read:
        lines = file_read.readlines()

    sentences = []
    sentence = []

    for line in lines:

        if line == "\n":
            if len(sentence) > 0:
                if 'DOCSTART' not in sentence[0][0]:
                    sentences.append(sentence)
                sentence = []
        else:
            word = line.split()
            sentence.append(word)
    return sentences

In [12]:
train_data = load_dataset("eng.train")
val_data = load_dataset("eng.testa")
test_data= load_dataset("eng.testb")

In [13]:
#sort out all different types of BIO tags
tags_list = []
for sentence in train_data:
  for word in sentence:
    if word != '\n':
        tags_list.append(word[-1])

In [14]:
label_set = list(set(tags_list))

In [15]:
label_set

['I-ORG', 'B-MISC', 'I-PER', 'I-LOC', 'B-ORG', 'B-LOC', 'I-MISC', 'O']

In [16]:
label_to_id_dict = {label:i for i, label in enumerate(label_set)}
id_to_label_dict = {value:key for key, value in label_to_id_dict.items()}

In [17]:
id_to_label_dict

{0: 'I-ORG',
 1: 'B-MISC',
 2: 'I-PER',
 3: 'I-LOC',
 4: 'B-ORG',
 5: 'B-LOC',
 6: 'I-MISC',
 7: 'O'}

In [18]:
#from text file, extract tokens and labels (converted into nums)
def extract_tokens_labels (content, label_dict):
    extract_dict = {"tokens":[], "labels":[]}
    # a single sentence as a value
    for sentence in content:
        sentence_words = []
        sent_label = []
        for word in sentence:
            sentence_words.append(word[0])
            sent_label.append(label_dict[word[-1]])
        extract_dict["tokens"].append(sentence_words)
        extract_dict["labels"].append(sent_label)
    return Dataset.from_dict(extract_dict)

In [19]:
train_dataset = extract_tokens_labels(train_data, label_to_id_dict)
val_dataset = extract_tokens_labels(val_data, label_to_id_dict)
test_dataset = extract_tokens_labels(test_data, label_to_id_dict)

In [20]:
datasets = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset,
})

In [21]:
model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_set))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
tokenizer

BertTokenizerFast(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [23]:
#Here, for the sub tokens that come from the same word, we just assign -100, so that they don't contribute heavily to loss
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True, padding=True
    )
    #after tokenizing, there might be more tokens than before tokenizing. Here, we align tokens and labels

    labels = []
    #the labels corresponding to the tokens
    for i, label in enumerate(examples["labels"]):
        #word_ids return the position of the words in the original sentence.multiple tokens could come from the same position after being tokenized into sub tokens
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # meaning that it could be a special token: [CLS] (start of sentence), [SEP] (end of sentence), [PAD]
            if word_idx is None:
                label_ids.append(-100)
            #the sub-token doesn't come from the the same word as the previous sub-token, then we just take the corresponding label (look at the position of the word)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            #the sub-token comes from the the same word as the previous sub-token, append -100 as label_id to label
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [24]:
datasets = datasets.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [25]:
metric = evaluate.load("seqeval")

In [26]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_set[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_set[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [27]:
data_collator = DataCollatorForTokenClassification(tokenizer = tokenizer, padding = 'longest', return_tensors = 'pt')

In [28]:
train_dataset

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 14041
})

In [29]:
from huggingface_hub import notebook_login

notebook_login()

In [30]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [31]:
def model_init():
    return AutoModelForTokenClassification.from_pretrained(model_name, id2label=id_to_label_dict, label2id=label_to_id_dict, num_labels=len(label_set))

In [36]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)


In [37]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.045400,0.037920,0.934071,0.937058,0.935562,0.989954
2,0.019900,0.035799,0.952493,0.954897,0.953694,0.992446
3,0.011000,0.036067,0.952900,0.956749,0.954820,0.992757


TrainOutput(global_step=5268, training_loss=0.03853298383132019, metrics={'train_runtime': 1248.3892, 'train_samples_per_second': 33.742, 'train_steps_per_second': 4.22, 'total_flos': 2694549004634928.0, 'train_loss': 0.03853298383132019, 'epoch': 3.0})

In [38]:
trainer.push_to_hub(commit_message="Training complete")

CommitInfo(commit_url='https://huggingface.co/jh-hoo/bert-finetuned-ner/commit/dce6d43ba9b9b277055bb41b0f7122899f42bc14', commit_message='Training complete', commit_description='', oid='dce6d43ba9b9b277055bb41b0f7122899f42bc14', pr_url=None, pr_revision=None, pr_num=None)

In [39]:
trainer.evaluate(eval_dataset = datasets["test"])

{'eval_loss': 0.13418059051036835,
 'eval_precision': 0.9075468394326738,
 'eval_recall': 0.9176699716713881,
 'eval_f1': 0.9125803327757724,
 'eval_accuracy': 0.9833746096694304,
 'eval_runtime': 25.5411,
 'eval_samples_per_second': 135.194,
 'eval_steps_per_second': 16.914,
 'epoch': 3.0}

Alt: If we were to use evaluator() on test set,there might be a certain format for test dataset to follow

from evaluate import evaluator
task_evaluator = evaluator("token-classification")

results = task_evaluator.compute(
    model_or_pipeline="jh-hoo/bert-finetuned-ner",
    data = test_eval,
    metric="seqeval",
)